In [1]:
import cv2
import numpy as np
from PIL import Image

import importlib

import py_vncorenlp
import torch
from transformers import AutoModel, AutoTokenizer

import preprocess_img
import LineOCR
importlib.reload(LineOCR)

import pandas as pd

import graph
from torch_geometric.utils.convert import from_networkx
import torch_geometric
importlib.reload(graph)

import torch.nn as nn
from torch_geometric.nn import ChebConv, GCNConv
import torch.nn.functional as F

import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu").type


ID2LABEL_DICT = {
    0: "None",
    1: "sign_date",
    2: "diagnose",
    3:"hospital_name",
    4:"address",
    5:"age",
    6:"treatment",
    7:"patient_name",
    8:"admission_date",
    9:"discharge_date",
    10:"gender",
    11:"document_type",
    12:"department",
    13:"note",
    14:"BHYT",
}

INFO 2022-10-19 17:38:28,334 utils.py:147] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
#Load Text Segmentation engine
# rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=r'D:\PPYCode\OCR\ppy_paddle\weights\nlp\vncorenlp')
# phobert = AutoModel.from_pretrained("vinai/phobert-base")
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
# %cd ../../../

In [3]:
#Load all models
preprocessImage = preprocess_img.PreprocessImage()
lineDetAndOCR = LineOCR.ProcessImage()

In [4]:
test_img = cv2.imread(r"imgs_test\705.jpeg")
test_img = preprocessImage(test_img)
# lines, t = lineDetInfer.line_det_infer(test_img)

In [5]:
# image = test_img.copy()
# for line in lines:
#     pts = line.astype(int)
#     pts = pts.reshape((-1, 1, 2))
    
#     isClosed = True
    
#     # Blue color in BGR
#     color = (255, 0, 0)
    
#     # Line thickness of 2 px
#     thickness = 2
    
#     image = cv2.polylines(image, [pts],
#                         isClosed, color, thickness)

# display(Image.fromarray(image))

In [6]:
result = lineDetAndOCR.begin_recognize_text(test_img)

35


In [7]:
import kie_gcn
from kie_gcn import InvoiceGCN
kieGCN = kie_gcn.KieGCN()

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


d:\PPYCode\OCR\ppy_paddle


In [8]:
kie_df = kieGCN(result, test_img)

In [11]:
kie_df

,index,xmin,ymin,xmax,ymax,Object,pred_label,confidence_score
0,0,128,43,343,69,BỆNH VIỆN QUÂN Y 211,hospital_name,0.965612
1,1,159,66,294,84,Khoa T-M-H (B9),department,0.989540
2,2,343,46,702,67,CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM,None,0.999201
3,3,423,66,638,86,Độc lập - Tự do - Hạnh phúc,None,0.999880
4,4,749,46,850,64,MS: 01/BV-01,None,0.997714
5,5,750,65,905,85,Số bệnh án: 22.006530,None,0.978973
6,6,751,85,943,105,Mã y tế: BV_211.22011209,None,0.589870
7,7,76,134,225,155,- Họ tên người bệnh:,None,0.995059
8,8,263,135,487,155,NGUYỄN THỊ THU HƯỜNG,patient_name,0.995732
9,9,439,109,611,136,GIẤY RA VIỆN,document_type,0.996945


In [ ]:
# for iter in range(len(result["croped_pil_img"])):
#     line_img = result["croped_pil_img"][iter]
#     text = result["text"][iter]
#     ocr_score = result["ocr_score"][iter]
#     display(line_img)
#     print(f"{text} \t {ocr_score}")
#     print("---------------------------------")

In [8]:
list_xmin = []
list_ymin = []
list_xmax = []
list_ymax = []
list_Object = []

for iter in range(len(result["bbox"])):
    xmin, ymin, xmax, ymax = result["bbox"][iter]
    text = result["text"][iter]
    list_xmin.append(xmin)
    list_xmax.append(xmax)
    list_ymin.append(ymin)
    list_ymax.append(ymax)
    list_Object.append(text)
single_df = {
    "xmin": list_xmin,
    "xmax": list_xmax,
    "ymin": list_ymin,
    "ymax": list_ymax,
    "Object": list_Object,
    "labels": ["None"]*len(list_Object)
}

In [9]:
single_df = pd.DataFrame.from_dict(single_df)

In [10]:
single_df.to_excel("temp_data/temp_gcn/csv/temp.xlsx")
cv2.imwrite("temp_data/temp_gcn/img/temp.jpg", test_img)

True

In [12]:
def get_sentence_features(sentence):
    segmented_sentence = rdrsegmenter.word_segment(sentence)
    tokenized_sentence = torch.tensor([tokenizer.encode(segmented_sentence[0])])
    features = []
    with torch.no_grad():
        features = phobert(tokenized_sentence)
    return features

def make_sent_bert_features(text):
    features = get_sentence_features(text)
    return features[1][0].cpu().numpy()

In [ ]:
file = 'temp'
connect = graph.Grapher(file)
G,result, df = connect.graph_formation(export_graph=False)
df = connect.relative_distance(export_document_graph = False)
individual_data = from_networkx(G)

In [25]:
feature_cols = [
            "rd_b",
            "rd_r",
            "rd_t",
            "rd_l",
            "line_number",
            "n_upper",
            "n_alpha",
            "n_spaces",
            "n_numeric",
            "n_special",
        ]

In [26]:
def preprocess_inference_data(df, individual_data):
        text_features = []
        for _, row in df.iterrows():
            text_features.append(make_sent_bert_features(row["Object"]))
        text_features = np.asarray(text_features, dtype=np.float32)

        numeric_features = df[feature_cols].values.astype(np.float32)

        features = np.concatenate((numeric_features, text_features), axis=1)
        features = torch.tensor(features)

        for col in df.columns:
            try:
                df[col] = df[col].str.strip()
            except AttributeError as e:
                pass
        text = df["Object"].values
        individual_data.x = features

        individual_data.text = text

        test_list_of_graphs = []
        test_list_of_graphs.append(individual_data)

        test_one_data = ""
        test_one_data = torch_geometric.data.Batch.from_data_list(test_list_of_graphs)
        test_one_data.edge_attr = None
        return test_one_data

In [27]:

test_data = preprocess_inference_data(df, individual_data)
test_data.to(device)

DataBatch(edge_index=[2, 90], num_nodes=35, x=[35, 778], text=[1], batch=[35], ptr=[2])

In [81]:
class InvoiceGCN(nn.Module):
    def __init__(self, input_dim, chebnet=False, n_classes=5, dropout_rate=0.2, K=3):
        super().__init__()

        self.input_dim = input_dim
        self.n_classes = n_classes
        self.dropout_rate = dropout_rate

        if chebnet:
            self.conv1 = ChebConv(self.input_dim, 64, K=K)
            # self.conv2 = ChebConv(64, 32, K=K)
            self.conv3 = ChebConv(64, 64, K=K)
            self.conv4 = ChebConv(64, self.n_classes, K=K)
        else:
            self.conv1 = GCNConv(self.first_dim, 64, improved=True, cached=True)
            self.conv2 = GCNConv(64, 32, improved=True, cached=True)
            self.conv3 = GCNConv(32, 16, improved=True, cached=True)
            self.conv4 = GCNConv(16, self.n_classes, improved=True, cached=True)

    def forward(self, data):
        # for transductive setting with full-batch update
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        x = F.dropout(F.relu(self.conv1(x, edge_index, edge_weight)), p=self.dropout_rate, training=self.training)
        # x = F.dropout(F.relu(self.conv2(x, edge_index, edge_weight)), p=self.dropout_rate, training=self.training)
        x = F.dropout(F.relu(self.conv3(x, edge_index, edge_weight)), p=self.dropout_rate, training=self.training)
        x = self.conv4(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

In [82]:
model = torch.load(r"D:\PPYCode\OCR\ppy_paddle\weights\gcn\GCN_221017.pth")
model.to(device)

InvoiceGCN(
  (conv1): ChebConv(778, 64, K=3, normalization=sym)
  (conv3): ChebConv(64, 64, K=3, normalization=sym)
  (conv4): ChebConv(64, 15, K=3, normalization=sym)
)

In [84]:
y_preds= model(test_data)
max_idx_Y_pred= y_preds.max(dim=1)[1].cpu().numpy()
shorten_pred = [y_preds[idx][max_idx_Y_pred[idx]] for idx in range(len(max_idx_Y_pred))]
pred_probs = torch.exp(torch.tensor(shorten_pred))


In [86]:
lb = [ID2LABEL_DICT[each] for each in list(max_idx_Y_pred)]

In [87]:
df["pred_class"] = lb
df["confidence_score"] = pred_probs

In [88]:
new_df = df[["index", "xmin", "ymin", "xmax", "ymax", "Object", "pred_class", "confidence_score"]]

In [10]:
new_df["confidence_score"]

NameError: name 'new_df' is not defined

In [31]:
for idx, each in enumerate(y_preds):
    text = test_data.text[0][idx]
    pred_class = ID2LABEL_DICT[each]
    print(text + '\t' + pred_class)
    print("-----------------")


BỆNH VIỆN QUÂN Y 211	hospital_name
-----------------
Khoa T-M-H (B9)	department
-----------------
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM	None
-----------------
Độc lập - Tự do - Hạnh phúc	None
-----------------
MS: 01/BV-01	None
-----------------
Số bệnh án: 22.006530	None
-----------------
Mã y tế: BV_211.22011209	None
-----------------
- Họ tên người bệnh:	None
-----------------
NGUYỄN THỊ THU HƯỜNG	patient_name
-----------------
GIẤY RA VIỆN	document_type
-----------------
Năm sinh: 2009	age
-----------------
Giới tính: Nữ	gender
-----------------
- Dân tộc : Kinh	None
-----------------
Nghề nghiệp: Sinh viên, học sinh	gender
-----------------
- Thẻ BHYT số: HS464642190886964002	BHYT
-----------------
giá trị từ:	None
-----------------
1/1/2022	None
-----------------
đến 31/12/2022	address
-----------------
- Vào viện lúc:	None
-----------------
Địa chỉ: Thôn Ia Lôk, Xã Ia Mơ Nông, Huyện Chư Păh, Tỉnh Gia Lai	address
-----------------
09	diagnose
-----------------
giờ 54 phút, ngày 11 t

# End2End inference

In [1]:
import cv2

import preprocess_img
import LineOCR
import pandas as pd
import numpy as np
import kie_gcn
from kie_gcn import InvoiceGCN

INFO 2022-10-19 19:51:37,863 utils.py:147] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
preprocessImage = preprocess_img.PreprocessImage()
lineDetAndOCR = LineOCR.ProcessImage()
kieGCN = kie_gcn.KieGCN()

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
test_img = cv2.imread(r"D:\PPYData\rotated_imgs\523.jpeg")
test_img = preprocessImage(test_img)

result = lineDetAndOCR.begin_recognize_text(test_img)
kie_df = kieGCN(result, test_img)

extracted_line time: 0.39733147621154785
31
ocr_line time: 2.150866746902466
make graph:  0.6225330829620361
transform:  4.214994192123413


In [4]:
kie_df

,index,xmin,ymin,xmax,ymax,Object,pred_label,confidence_score
0,0,139,128,431,152,QUÂN ĐỘI NHÂN DÂN VIỆT NAM,None,0.989248
1,1,451,128,959,151,CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM MS: 01/BV-01,None,0.977155
2,2,137,161,336,180,BỆNH VIỆN QUÂN Y 15,hospital_name,0.998870
3,3,509,161,764,184,Độc lập - Tự do - Hạnh Phúc,None,0.998809
4,4,839,158,1011,179,Số lưu trữ: 22000072,None,0.997743
5,5,137,190,412,212,Khoa YHCT Và PHCN BHXH,department,0.999105
6,6,839,185,1069,209,Mã Y tế: 064/246/22/0000,None,0.999517
7,7,531,248,680,270,GIẤY RA VIỆN,document_type,0.998910
8,8,136,280,617,307,-Họ tên người bệnh: NGUYỄN THỊ NGỌC HƯƠNG,patient_name,0.999724
9,9,689,278,1026,309,Ngày sinh: 28/07/1986 Nam/Nữ: Nữ,age,0.836389


In [8]:
from concurrent.futures.thread import ThreadPoolExecutor
import time

In [ ]:
start = time()

processes = []
#743414
with ThreadPoolExecutor(max_workers=50) as executor:
    for page in range(70000,74342):
        processes.append(executor.submit(request_short_data, page))
total_responds = []
for task in tqdm(as_completed(processes), total = len(processes)):
    total_responds+= task.result()
print(f'Time taken: {time() - start}')